In [1]:
DF = sqlContext.read.format('com.databricks.spark.csv').options(delimiter=',', header='true', inferschema='true').load("dbfs:/mnt/s3/data/train_v2_flatten.csv")
DF.cache()

Out[ 1 ]: DataFrame[channelGrouping: string, date: int, fullVisitorId: string, socialEngagementType: string, visitId: string, visitNumber: string, visitStartTime: string, device_browser: string, device_browserSize: string, device_browserVersion: string, device_deviceCategory: string, device_flashVersion: string, device_isMobile: boolean, device_language: string, device_mobileDeviceBranding: string, device_mobileDeviceInfo: string, device_mobileDeviceMarketingName: string, device_mobileDeviceModel: string, device_mobileInputSelector: string, device_operatingSystem: string, device_operatingSystemVersion: string, device_screenColors: string, device_screenResolution: string, geoNetwork_city: string, geoNetwork_cityId: string, geoNetwork_continent: string, geoNetwork_country: string, geoNetwork_latitude: string, geoNetwork_longitude: string, geoNetwork_metro: string, geoNetwork_networkDomain: string, geoNetwork_networkLocation: string, geoNetwork_region: string, geoNetwork_subContinent: string, totals_bounces: int, totals_hits: int, totals_newVisits: int, totals_pageviews: string, totals_sessionQualityDim: string, totals_timeOnSite: string, totals_totalTransactionRevenue: bigint, totals_transactionRevenue: string, totals_transactions: string, totals_visits: string, trafficSource_adContent: string, trafficSource_campaign: string, trafficSource_campaignCode: string, trafficSource_isTrueDirect: string, trafficSource_keyword: string, trafficSource_medium: string, trafficSource_referralPath: string, trafficSource_source: string, trafficSource_adwordsClickInfo_adNetworkType: string, trafficSource_adwordsClickInfo_criteriaParameters: string, trafficSource_adwordsClickInfo_gclId: string, trafficSource_adwordsClickInfo_isVideoAd: boolean, trafficSource_adwordsClickInfo_page: int, trafficSource_adwordsClickInfo_slot: string, trafficSource_adwordsClickInfo_targetingCriteria: string, customDimensions_index: string, customDimensions_value: string, hits_dataSource: string, hits_eventInfo: string, hits_hitNumber: int, hits_hour: string, hits_isEntrance: boolean, hits_isExit: string, hits_isInteraction: string, hits_latencyTracking: string, hits_minute: int, hits_referer: string, hits_time: string, hits_type: string, hits_page_hostname: string, hits_page_pagePath: string, hits_page_pagePathLevel1: string, hits_page_pagePathLevel2: string, hits_page_pagePathLevel3: string, hits_page_pagePathLevel4: string, hits_page_pageTitle: string, hits_page_searchCategory: string, hits_page_searchKeyword: string, hits_page_0: string, hits_transaction_currencyCode: string, hits_transaction_0: string, hits_transaction_affiliation: string, hits_transaction_localTransactionRevenue: int, hits_transaction_localTransactionShipping: int, hits_transaction_localTransactionTax: int, hits_transaction_transactionId: string, hits_transaction_transactionRevenue: int, hits_transaction_transactionShipping: int, hits_transaction_transactionTax: int, hits_item_currencyCode: string, hits_item_0: string, hits_item_transactionId: string, hits_appInfo_exitScreenName: string, hits_appInfo_landingScreenName: string, hits_appInfo_screenDepth: int, hits_appInfo_screenName: string, hits_appInfo_0: string, hits_exceptionInfo_isFatal: boolean, hits_exceptionInfo_0: string, hits_eCommerceAction_action_type: int, hits_eCommerceAction_step: int, hits_eCommerceAction_0: string, hits_eCommerceAction_option: string, hits_social_hasSocialSourceReferral: string, hits_social_socialInteractionNetworkAction: string, hits_social_socialNetwork: string, hits_social_0: string, hits_contentGroup_contentGroup1: string, hits_contentGroup_contentGroup2: string, hits_contentGroup_contentGroup3: string, hits_contentGroup_contentGroup4: string, hits_contentGroup_contentGroup5: string, hits_contentGroup_contentGroupUniqueViews1: int, hits_contentGroup_contentGroupUniqueViews2: int, hits_contentGroup_contentGroupUniqueViews3: int, hits_contentGroup_previousContentGroup1: string, hits_contentGroup_previousContentGr

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
def deleteNull_castToFloat(x):
  
  if x == None:
    return 0.0
  if x == '(not set)':
    return 0
  if x == 'No':
    return 0.0
  else:
    y = len(x)
    if y>6:
      i = int(x[ : y-6])
      f = int(x[y-6:y])
    else:
      i = 0
      f = int(x)
    a = float(i)
    b = float(f)/1000000
    return a+b
udf_deleteNull_castToFloat = F.udf(deleteNull_castToFloat, T.FloatType())

def deleteNull_castToInt(x):
  
  if x == None:
    return 0
  if x == '(not set)':
    return 0
  if x == 'No':
    return 0
  else:
    return int(x)
udf_deleteNull_castToInt = F.udf(deleteNull_castToInt, T.IntegerType())

def process_timeOnSite(x):
  if x == None:
    return 0
  elif x.isdigit():
    return int(x)
  else:
    return 0
udf_process_timeOnSite = F.udf(process_timeOnSite, T.IntegerType())

def process_totalTR(x):
  if x == None:
    return 0.0
  else:
    return (x/1000000.0)
udf_process_totalTR = F.udf(process_totalTR, T.FloatType())

def deleteNull_castToInt_visitNumber(x):
  
  if x == None:
    return 0
  if x == '(not set)':
    return 0
  if x == 'No':
    return 0
  if len(x)>10:
    return 0
  else:
    return int(x)
udf_deleteNull_castToInt_visitNumber = F.udf(deleteNull_castToInt_visitNumber, T.IntegerType())

In [3]:
display(DF)

channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserSize,device_browserVersion,device_deviceCategory,device_flashVersion,device_isMobile,device_language,device_mobileDeviceBranding,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_mobileDeviceModel,device_mobileInputSelector,device_operatingSystem,device_operatingSystemVersion,device_screenColors,device_screenResolution,geoNetwork_city,geoNetwork_cityId,geoNetwork_continent,geoNetwork_country,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_networkLocation,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,totals_visits,trafficSource_adContent,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_gclId,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_targetingCriteria,customDimensions_index,customDimensions_value,hits_dataSource,hits_eventInfo,hits_hitNumber,hits_hour,hits_isEntrance,hits_isExit,hits_isInteraction,hits_latencyTracking,hits_minute,hits_referer,hits_time,hits_type,hits_page_hostname,hits_page_pagePath,hits_page_pagePathLevel1,hits_page_pagePathLevel2,hits_page_pagePathLevel3,hits_page_pagePathLevel4,hits_page_pageTitle,hits_page_searchCategory,hits_page_searchKeyword,hits_page_0,hits_transaction_currencyCode,hits_transaction_0,hits_transaction_affiliation,hits_transaction_localTransactionRevenue,hits_transaction_localTransactionShipping,hits_transaction_localTransactionTax,hits_transaction_transactionId,hits_transaction_transactionRevenue,hits_transaction_transactionShipping,hits_transaction_transactionTax,hits_item_currencyCode,hits_item_0,hits_item_transactionId,hits_appInfo_exitScreenName,hits_appInfo_landingScreenName,hits_appInfo_screenDepth,hits_appInfo_screenName,hits_appInfo_0,hits_exceptionInfo_isFatal,hits_exceptionInfo_0,hits_eCommerceAction_action_type,hits_eCommerceAction_step,hits_eCommerceAction_0,hits_eCommerceAction_option,hits_social_hasSocialSourceReferral,hits_social_socialInteractionNetworkAction,hits_social_socialNetwork,hits_social_0,hits_contentGroup_contentGroup1,hits_contentGroup_contentGroup2,hits_contentGroup_contentGroup3,hits_contentGroup_contentGroup4,hits_contentGroup_contentGroup5,hits_contentGroup_contentGroupUniqueViews1,hits_contentGroup_contentGroupUniqueViews2,hits_contentGroup_contentGroupUniqueViews3,hits_contentGroup_previousContentGroup1,hits_contentGroup_previousContentGroup2,hits_contentGroup_previousContentGroup3,hits_contentGroup_previousContentGroup4,hits_contentGroup_previousContentGroup5,hits_contentGroup_0,hits_promotionActionInfo_0,hits_promotionActionInfo_promoIsView,hits_promotionActionInfo_promoIsClick,hits_product_customDimensions,hits_product_customMetrics,hits_product_isClick,hits_product_isImpression,hits_product_localProductPrice,hits_product_localProductRevenue,hits_product_productBrand,hits_product_productCouponCode,hits_product_productListName,hits_product_productListPosition,hits_product_productPrice,hits_product_productQuantity,hits_product_productRevenue,hits_product_productSKU,hits_product_productVariant,hits_product_v2ProductCategory,hits_product_v2ProductName,hits_promotion_promoCreative,hits_promotion_promoId,hits_promotion_promoName,hits_promotion_promoPosition
Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,false,not available

In [4]:
from pyspark.sql.functions import col
DF_select = DF.select(udf_deleteNull_castToFloat("totals_transactionRevenue").alias("revenue"), udf_process_totalTR("totals_totalTransactionRevenue").alias("totalTR"), "device_operatingSystem", "device_browser",  "geoNetwork_country", "channelGrouping", (col('visitStartTime')).cast("string").alias("startTime_cast"), (col('date')).cast("string").alias("date_cast"), "fullVisitorId", udf_deleteNull_castToInt("totals_hits").alias("hits"), udf_deleteNull_castToInt("totals_pageviews").alias("pageview"), udf_deleteNull_castToInt_visitNumber("visitNumber").alias("visitNumber"), udf_process_timeOnSite("totals_timeOnSite").alias("timeOnSite"), udf_process_timeOnSite("totals_transactions").alias("totals_transactions"))

In [5]:
display(DF_select)

revenue,totalTR,device_operatingSystem,device_browser,geoNetwork_country,channelGrouping,startTime_cast,date_cast,fullVisitorId,hits,pageview,visitNumber,timeOnSite,totals_transactions
0.0,0.0,Windows,Firefox,Germany,Organic Search,1508198450,20171016,3162355547410993243,1,1,1,0,0
0.0,0.0,Chrome OS,Chrome,United States,Referral,1508176307,20171016,8934116514970143966,2,2,6,28,0
0.0,0.0,Android,Chrome,United States,Direct,1508201613,20171016,7992466427990357681,2,2,1,38,0
0.0,0.0,Windows,Chrome,Turkey,Organic Search,1508169851,20171016,9075655783635761930,2,2,1,1,0
0.0,0.0,Windows,Chrome,Mexico,Organic Search,1508190552,20171016,6960673291025684308,2,2,1,52,0
0.0,0.0,Macintosh,Chrome,United States,Referral,1508196701,20171016,0166277907528479249,2,2,1,12,0
0.0,0.0,Macintosh,Chrome,United Kingdom,Referral,1508152478,20171016,8349655975937271469,2,2,1,9,0
0.0,0.0,Windows,Chrome,Denmark,Organic Search,1508206208,20171016,1332629902468998662,2,2,1,15,0
0.0,0.0,Macintosh,Chrome,Mexico,Organic Search,1508207516,20171016,632878546807742341,2,2,1,34,0
0.0,0.0,iOS,Safari,Netherlands,Organic Search,1508165159,20171016,1259490915281096752,2,2,2,32,0


In [6]:
from pyspark.sql.functions import unix_timestamp, to_date
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import month, dayofweek, hour, year, weekofyear, dayofyear

DF_select = DF_select.withColumn('date_cast', to_date('date_cast', 'yyyyMMdd'))
#DF_select = DF_select.withColumn("month", month("date_cast"))
#DF_select = DF_select.withColumn("weekday", dayofweek("date_cast"))
#DF_select = DF_select.withColumn("year", year("date_cast"))
DF_select = DF_select.withColumn("yearweek", weekofyear("date_cast"))
DF_select = DF_select.withColumn("yearday", dayofyear("date_cast"))
#DF_select = DF_select.withColumn("startTime_timestamp", from_unixtime("startTime_cast"))
#DF_select = DF_select.withColumn("hour", hour("startTime_timestamp"))
#DF_select = DF_select.drop('startTime_timestamp', "startTime_cast", 'date_cast')
DF_select.cache()

Out[ 5 ]: DataFrame[revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, fullVisitorId: string, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int]

In [7]:
DF_fea = DF_select

In [8]:
# process geoNetwork_country
import numpy as np
country_5k_10k = DF_select.groupBy("geoNetwork_country").count().filter("count>5000").filter("count<10000").select('geoNetwork_country')
country_0_5k = DF_select.groupBy("geoNetwork_country").count().filter("count<=5000").select('geoNetwork_country')
country_5k_10k = np.array(country_5k_10k.toPandas().geoNetwork_country)
country_0_5k = np.array(country_0_5k.toPandas().geoNetwork_country)

def process_country(x):
  if x == None:
    return 'Other'
  elif x == '(not set)':
    return 'Other'
  elif x in country_0_5k:
    return 'Other_0_5k'
  elif x in country_5k_10k:
    return 'Other_5k_10k'
  else:
    return x
  
  
udf_process_country = F.udf(process_country, T.StringType())
DF_fea = DF_fea.withColumn('process_country', udf_process_country('geoNetwork_country'))
DF_fea.cache()

Out[ 7 ]: DataFrame[revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, fullVisitorId: string, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int, process_country: string]

In [9]:
#Generating features grouped by dayofyear & weekofyear

DF_yearday = DF_fea.groupby('yearday').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_day').withColumnRenamed("yearday", 'yearday2')
DF_fea = DF_fea.join(DF_yearday, DF_yearday.yearday2 == DF_fea.yearday).drop('yearday2')


DF_yearweek = DF_fea.groupby('yearweek').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_week').withColumnRenamed("yearweek", 'yearweek2')
DF_fea = DF_fea.join(DF_yearweek, DF_yearweek.yearweek2 == DF_fea.yearweek).drop('yearweek2')
DF_fea.cache()

Out[ 8 ]: DataFrame[revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, fullVisitorId: string, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int, process_country: string, sum_hits_per_day: bigint, sum_hits_per_week: bigint]

In [10]:
#Generating features grouped by geoNetwork_country

DF_fea = DF_fea.join(DF_fea.groupby('process_country').sum('pageview').withColumnRenamed('sum(pageview)', 'sum_pageviews_per_country'), "process_country")

#DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_country').avg('pageview').withColumnRenamed('avg(pageview)', 'avg_pageviews_per_country'), "geoNetwork_country")

#DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_country').agg(F.count('pageview')).withColumnRenamed('count(pageview)', 'count_pageviews_per_country'),"geoNetwork_country")


DF_fea = DF_fea.join(DF_fea.groupby('process_country').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_country'), "process_country")

#DF_fea = DF_fea.join(DF_fea.groupby('process_country').avg('hits').withColumnRenamed('avg(hits)', 'avg_hits_per_country'), "process_country")

#DF_fea = DF_fea.join(DF_fea.groupby('process_country').agg(F.count('hits')).withColumnRenamed('count(hits)', 'count_hits_per_country'),"process_country")
DF_fea.cache()

Out[ 9 ]: DataFrame[process_country: string, revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, fullVisitorId: string, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int, sum_hits_per_day: bigint, sum_hits_per_week: bigint, sum_pageviews_per_country: bigint, sum_hits_per_country: bigint]

In [11]:
#Generating features grouped by fullVisitorId

DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').sum('pageview').withColumnRenamed('sum(pageview)', 'sum_pageviews_per_fullVisitorId'), "fullVisitorId")

#DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').avg('pageview').withColumnRenamed('avg(pageview)', 'avg_pageviews_per_fullVisitorId'), "fullVisitorId")

#DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').agg(F.count('pageview')).withColumnRenamed('count(pageview)', 'count_pageviews_per_fullVisitorId'),"fullVisitorId")

DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_fullVisitorId'), "fullVisitorId")

#DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').avg('hits').withColumnRenamed('avg(hits)', 'avg_hits_per_fullVisitorId'), "fullVisitorId")

#DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').agg(F.count('hits')).withColumnRenamed('count(hits)', 'count_hits_per_fullVisitorId'),"fullVisitorId")
DF_fea.cache()

Out[ 10 ]: DataFrame[fullVisitorId: string, process_country: string, revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int, sum_hits_per_day: bigint, sum_hits_per_week: bigint, sum_pageviews_per_country: bigint, sum_hits_per_country: bigint, sum_pageviews_per_fullVisitorId: bigint, sum_hits_per_fullVisitorId: bigint]

In [12]:
DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').avg('timeOnSite').withColumnRenamed('avg(timeOnSite)', 'avg_timeOnSite_per_fullVisitorId'), "fullVisitorId")
DF_fea.cache()

Out[ 11 ]: DataFrame[fullVisitorId: string, process_country: string, revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int, sum_hits_per_day: bigint, sum_hits_per_week: bigint, sum_pageviews_per_country: bigint, sum_hits_per_country: bigint, sum_pageviews_per_fullVisitorId: bigint, sum_hits_per_fullVisitorId: bigint, avg_timeOnSite_per_fullVisitorId: double]

In [13]:
sqlContext.registerDataFrameAsTable(DF_fea, "DF")

In [14]:
%sql

insert into DF select sum_timeOnSite_per_fullVisitorId/visitNumber as timeOnSite_to_visitNumber from DF

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: cannot resolve '`sum_timeOnSite_per_fullVisitorId`' given input columns: [df.startTime_cast, df.fullVisitorId, df.device_operatingSystem, df.timeOnSite, df.yearday, df.sum_hits_per_fullVisitorId, df.geoNetwork_networkDomain, df.sum_hits_per_country, df.sum_hits_per_week, df.channelGrouping, df.geoNetwork_country, df.avg(timeOnSite), df.revenue, df.yearweek, df.sum_pageviews_per_fullVisitorId, df.device_browser, df.totalTR, df.visitNumber, df.hits, df.geoNetwork_region, df.sum_hits_per_day, df.date_cast, df.totals_transactions, df.process_country, df.pageview, df.sum_pageviews_per_country]; line 1 pos 22;
'InsertIntoTable 'UnresolvedRelation `DF`, false, false
+- 'Project [('sum_timeOnSite_per_fullVisitorId / visitNumber#51555) AS timeOnSite_to_visitNumber#59830]
 +- SubqueryAlias df
 +- Project [fullVisitorId#15472, process_country#52026, revenue#51549, totalTR#51550, device_operatingSystem#15489, device_browser#15477, geoNetwork_country#15496, channelGrouping#15470, startTime_cast#51551, date_cast#51614, hits#51553, pageview#51554, visitNumber#51555, geoNetwork_networkDomain#15500, geoNetwork_region#15502, timeOnSite#51556, totals_transactions#51557, yearweek#51631, yearday#51649, sum_hits_per_day#52193L, sum_hits_per_week#52306L, sum_pageviews_per_country#52424L, sum_hits_per_country#52479L, sum_pageviews_per_fullVisitorId#52536L, ... 2 more fields]
 +- Join Inner, (fullVisitorId#15472 = fullVisitorId#57420)
 :- Project [fullVisitorId#15472, process_country#52026, revenue#51549, totalTR#51550, device_operatingSystem#15489, device_browser#15477, geoNetwork_country#15496, channelGrouping#15470, startTime_cast#51551, date_cast#51614, hits#51553, pageview#51554, visitNumber#51555, geoNetwork_networkDomain#15500, geoNetwork_region#15502, timeOnSite#51556, totals_transactions#51557, yearweek#51631, yearday#51649, sum_hits_per_day#52193L, sum_hits_per_week#52306L, sum_pageviews_per_country#52424L, sum_hits_per_country#52479L, sum_pageviews_per_fullVisitorId#52536L, sum_hits_per_fullVisitorId#54975L]
 : +- Join Inner, (fullVisitorId#15472 = fullVisitorId#54980)
 : :- Project [fullVisitorId#15472, process_country#52026, revenue#51549, totalTR#51550, device_operatingSystem#15489, device_browser#15477, geoNetwork_country#15496, channelGrouping#15470, startTime_cast#51551, date_cast#51614, hits#51553, pageview#51554, visitNumber#51555, geoNetwork_networkDomain#15500, geoNetwork_region#15502, timeOnSite#51556, totals_transactions#51557, yearweek#51631, yearday#51649, sum_hits_per_day#52193L, sum_hits_per_week#52306L, sum_pageviews_per_country#52424L, sum_hits_per_country#52479L, sum_pageviews_per_fullVisitorId#52536L]
 : : +- Join Inner, (fullVisitorId#15472 = fullVisitorId#52541)
 : : :- Project [process_country#52026, revenue#51549, totalTR#51550, device_operatingSystem#15489, device_browser#15477, geoNetwork_country#15496, channelGrouping#15470, startTime_cast#51551, date_cast#51614, fullVisitorId#15472, hits#51553, pageview#51554, visitNumber#51555, geoNetwork_networkDomain#15500, geoNetwork_region#15502, timeOnSite#51556, totals_transactions#51557, yearweek#51631, yearday#51649, sum_hits_per_day#52193L, sum_hits_per_week#52306L, sum_pageviews_per_country#52424L, sum_hits_per_country#52479L]
 : : : +- Join Inner, (process_country#52026 = process_country#52482)
 : : : :- Project [process_country#52026, revenue#51549, totalTR#51550, device_operatingSystem#15489, device_browser#15477, geoNetwork_country#15496, channelGrouping#15470, startTime_cast#51551, date_cast#51614, fullVisitorId#15472, hits#51553, pageview#51554, visitNumber#51555, geoNetwork_networkDomain#15500, geoNetwork_region#15502, timeOnSite#51556, totals_transactions#51557, yearweek#51631, yearday#51649, sum_hits_per_day#52193L, sum_hits_per_week#52306L, sum_pageviews_per_country#52424L]
 : : : : +- Join Inner, (process_country#52026 = process_country#52427

In [15]:
DF_select_group = DF_select.groupby('fullVisitorId').avg('visitNumber')

In [16]:
display(DF_select.join(DF_select_group,DF_select.fullVisitorId == DF_select_group.fullVisitorId))

In [17]:
# process geoNetwork_country
import numpy as np
country_5k_10k = DF_select.groupBy("geoNetwork_country").count().filter("count>5000").filter("count<10000").select('geoNetwork_country')
country_0_5k = DF_select.groupBy("geoNetwork_country").count().filter("count<=5000").select('geoNetwork_country')
country_5k_10k = np.array(country_5k_10k.toPandas().geoNetwork_country)
country_0_5k = np.array(country_0_5k.toPandas().geoNetwork_country)

def process_country(x):
  if x == None:
    return 'Other'
  elif x == '(not set)':
    return 'Other'
  elif x in country_0_5k:
    return 'Other_0_5k'
  elif x in country_5k_10k:
    return 'Other_5k_10k'
  else:
    return x
  
  
udf_process_country = F.udf(process_country, T.StringType())
DF_fea = DF_fea.withColumn('process_country', udf_process_country('geoNetwork_country'))
display(DF_fea.groupBy("process_country").agg(F.count("process_country")).orderBy("count(process_country)"))
                      

In [18]:
country_list = np.array(DF_fea.groupBy('process_country').count().orderBy('count').toPandas().process_country)
country_list

Out[ 12 ]: 
array([u'Other', u'Malaysia', u'South Korea', u'Ireland', u'Singapore',
 u'Philippines', u'Poland', u'Indonesia', u'Russia', u'Italy',
 u'Netherlands', u'Spain', u'Australia', u'Mexico', u'Taiwan',
 u'Turkey', u'Thailand', u'France', u'Vietnam', u'Brazil', u'Japan',
 u'Germany', u'Canada', u'United Kingdom', u'India', u'Other_0_5k',
 u'Other_5k_10k', u'United States'], dtype=object)

In [19]:
# process OS
OS_0_10k = DF_select.groupBy("device_operatingSystem").count().filter("count<10000").select('device_operatingSystem')
OS_0_10k = np.array(OS_0_10k.toPandas().device_operatingSystem)


def process_OS(x):
  if x == None:
    return 'Other'
  elif x == '(not set)':
    return 'Other'
  elif x in OS_0_10k:
    return 'Other_0_10k'
  else:
    return x
  
udf_process_OS = F.udf(process_OS, T.StringType())
DF_fea = DF_fea.withColumn('process_OS', udf_process_OS('device_operatingSystem'))
DF_fea.cache()

Out[ 13 ]: DataFrame[fullVisitorId: string, process_country: string, revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int, sum_hits_per_day: bigint, sum_hits_per_week: bigint, sum_pageviews_per_country: bigint, sum_hits_per_country: bigint, sum_pageviews_per_fullVisitorId: bigint, sum_hits_per_fullVisitorId: bigint, avg_timeOnSite_per_fullVisitorId: double, process_OS: string]

In [20]:
OS_list = np.array(DF_fea.groupBy('process_OS').count().orderBy('count').toPandas().process_OS)
OS_list

Out[ 14 ]: 
array([u'Other_0_10k', u'Other', u'Chrome OS', u'Linux', u'iOS',
 u'Android', u'Macintosh', u'Windows'], dtype=object)

In [21]:
# process geoNetwork_country
import numpy as np
browser_5k_10k = DF_select.groupBy("device_browser").count().filter("count>5000").filter("count<10000").select('device_browser')
browser_0_5k = DF_select.groupBy("device_browser").count().filter("count<=5000").select('device_browser')
browser_5k_10k = np.array(browser_5k_10k.toPandas().device_browser)
browser_0_5k = np.array(browser_0_5k.toPandas().device_browser)

def process_browser(x):
  if x == None:
    return 'Other'
  elif x == '(not set)':
    return 'Other'
  elif x in browser_5k_10k:
    return 'Other_5k_10k'
  elif x in browser_0_5k:
    return 'Other_0_5k'
  else:
    return x
  
udf_process_browser = F.udf(process_browser, T.StringType())
DF_fea = DF_fea.withColumn('process_browser', udf_process_browser('device_browser'))
DF_fea.cache()

Out[ 15 ]: DataFrame[fullVisitorId: string, process_country: string, revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int, sum_hits_per_day: bigint, sum_hits_per_week: bigint, sum_pageviews_per_country: bigint, sum_hits_per_country: bigint, sum_pageviews_per_fullVisitorId: bigint, sum_hits_per_fullVisitorId: bigint, avg_timeOnSite_per_fullVisitorId: double, process_OS: string, process_browser: string]

In [22]:
browser_list = np.array(DF_fea.groupBy('process_browser').count().orderBy('count').toPandas().process_browser)
browser_list

Out[ 16 ]: 
array([u'Other', u'Other_0_5k', u'Safari (in-app)', u'Opera Mini',
 u'Other_5k_10k', u'Samsung Internet', u'Edge', u'Android Webview',
 u'Internet Explorer', u'Firefox', u'Safari', u'Chrome'], dtype=object)

In [23]:
# process channelGrouping
def process_CG(x):
  if x == 'Affiliates':
    return 'Affiliates'
  if x == 'Paid Search':
    return 'Paid Search'
  if x == 'Display':
    return 'Display'
  if x == 'Referral':
    return 'Referral'
  if x == 'Direct':
    return 'Direct'
  if x == 'Social':
    return 'Social'
  if x == 'Organic Search':
    return 'Organic Search'
  else:
    return 'Other'
  
udf_process_CG = F.udf(process_CG, T.StringType())
DF_fea = DF_fea.withColumn('process_CG', udf_process_CG('channelGrouping'))
DF_fea.cache()

Out[ 17 ]: DataFrame[fullVisitorId: string, process_country: string, revenue: float, totalTR: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, hits: int, pageview: int, visitNumber: int, timeOnSite: int, totals_transactions: int, yearweek: int, yearday: int, sum_hits_per_day: bigint, sum_hits_per_week: bigint, sum_pageviews_per_country: bigint, sum_hits_per_country: bigint, sum_pageviews_per_fullVisitorId: bigint, sum_hits_per_fullVisitorId: bigint, avg_timeOnSite_per_fullVisitorId: double, process_OS: string, process_browser: string, process_CG: string]

In [24]:
CG_list = np.array(DF_fea.groupBy('process_CG').count().orderBy('count').toPandas().process_CG)
CG_list

Out[ 18 ]: 
array([u'Other', u'Affiliates', u'Paid Search', u'Display', u'Referral',
 u'Direct', u'Social', u'Organic Search'], dtype=object)

In [25]:
DF_fea.cache()
display(DF_fea)

fullVisitorId,process_country,revenue,totalTR,device_operatingSystem,device_browser,geoNetwork_country,channelGrouping,startTime_cast,date_cast,hits,pageview,visitNumber,timeOnSite,totals_transactions,yearweek,yearday,sum_hits_per_day,sum_hits_per_week,sum_pageviews_per_country,sum_hits_per_country,sum_pageviews_per_fullVisitorId,sum_hits_per_fullVisitorId,avg_timeOnSite_per_fullVisitorId,process_OS,process_browser,process_CG
000302540561900883,Singapore,0.0,0.0,iOS,Safari,Singapore,Organic Search,1481610074,2016-12-12,1,1,1,0,0,50,347,41818,219814,44091,52571,1,1,0.0,iOS,Safari,Organic Search
0003117951959211834,France,0.0,0.0,Windows,Chrome,France,Social,1502544078,2017-08-12,3,3,1,50,0,32,224,25007,175950,91686,106656,3,3,50.0,Windows,Chrome,Social
0003829380167140408,United States,0.0,0.0,Linux,Chrome,United States,Referral,1501288488,2017-07-28,8,8,1,139,0,30,209,11192,70239,3500525,4267861,8,8,139.0,Linux,Chrome,Referral
0004461751887554862,United States,0.0,0.0,Android,Chrome,United States,Organic Search,1513900985,2017-12-21,5,4,1,184,0,51,355,28033,150926,3500525,4267861,4,5,184.0,Android,Chrome,Organic Search
0006157756492474298,United States,0.0,0.0,Android,Android Webview,United States,Direct,1513672747,2017-12-19,1,1,1,0,0,51,353,25620,150926,3500525,4267861,2,2,0.0,Android,Android Webview,Direct
0006157756492474298,United States,0.0,0.0,Android,Android Webview,United States,Direct,1513832348,2017-12-20,1,1,2,0,0,51,354,29005,150926,3500525,4267861,2,2,0.0,Android,Android Webview,Direct
0007332370604472624,Taiwan,0.0,0.0,Windows,Chrome,Taiwan,Organic Search,1491105372,2017-04-01,37,27,1,613,0,13,91,16201,165057,98659,116957,27,37,613.0,Windows,Chrome,Organic Search
0007536034297456582,Indonesia,0.0,0.0,Linux,UC Browser,Indonesia,Direct,1489809978,2017-03-17,1,1,1,0,0,11,76,18310,152034,38101,43647,1,1,0.0,Linux,Other_5k_10k,Direct
0009038723733767042,United Kingdom,0.0,0.0,Windows,Chrome,United Kingdom,Organic Search,1484126244,2017-01-11,2,2,1,12,0,2,11,22389,144686,175051,198757,2,2,12.0,Windows,Chrome,Organic Search
0009771503731572214,United States,0.0,0.0,Macintosh,Chrome,United States,Referral,1472409907,2016-08-28,1,1,1,0,0,34,241,24600,186124,3500525,4267861,1,1,0.0,Macintosh,Chrome,Referral


In [26]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator
all_col = ['revenue', 'totalTR', 'process_CG', 'process_browser', 'process_country', 'process_OS', 'hits', 'pageview', 'visitNumber', 'avg_timeOnSite_per_fullVisitorId', 'sum_hits_per_day', 'sum_hits_per_week', 'sum_pageviews_per_country', 'sum_hits_per_country', 'sum_pageviews_per_fullVisitorId', 'sum_hits_per_fullVisitorId']
categorical_features = ['process_CG', 'process_browser', 'process_country', 'process_OS']
conti_features = ['totalTR', 'hits', 'pageview', 'visitNumber', 'avg_timeOnSite_per_fullVisitorId', 'sum_hits_per_day', 'sum_hits_per_week', 'sum_pageviews_per_country', 'sum_hits_per_country', 'sum_pageviews_per_fullVisitorId', 'sum_hits_per_fullVisitorId']
stages = [] # stages in our Pipeline
# One-hot encode cotegorical feature
for i in categorical_features:
  stringIndexer = StringIndexer(inputCol=i, outputCol=i + "_Index").setHandleInvalid('skip')
  #StringIndexer.handleInvalid('skip')
  encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[i + "_Vec"])
  stages += [stringIndexer, encoder]

In [27]:
assemblerInputs = [i + "_Vec" for c in categorical_features]+conti_features
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [28]:
DF_train = DF_fea.select('revenue', 'totalTR', 'process_CG', 'process_browser', 'process_country', 'process_OS', 'hits', 'pageview', 'visitNumber', 'avg_timeOnSite_per_fullVisitorId', 'sum_hits_per_day', 'sum_hits_per_week', 'sum_pageviews_per_country', 'sum_hits_per_country', 'sum_pageviews_per_fullVisitorId', 'sum_hits_per_fullVisitorId')
DF_train.cache()
display(DF_train)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2487822865328491> in <module> () 
 ----> 1 DF_train = DF_fea . select ( 'revenue' , 'totalTR' , 'process_CG' , 'process_browser' , 'process_country' , 'process_OS' , 'hits' , 'pageview' , 'visitNumber' , 'avg_timeOnSite_per_fullVisitorId' , 'sum_hits_per_day' , 'sum_hits_per_week' , 'sum_pageviews_per_country' , 'sum_hits_per_country' , 'sum_pageviews_per_fullVisitorId' , 'sum_hits_per_fullVisitorId' , 'features' ) 
 2 DF_train . cache ( ) 
 3 display ( DF_train ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1263 [ Row ( name = u'Alice' , age = 12 ) , Row ( name = u'Bob' , age = 15 ) ] 
 1264 """
 -> 1265 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1266 return DataFrame ( jdf , self . sql_ctx ) 
 1267 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : u"cannot resolve '`features`' given input columns: [process_browser, process_country, sum_pageviews_per_country, hits, yearday, channelGrouping, geoNetwork_country, device_browser, pageview, sum_hits_per_day, device_operatingSystem, sum_hits_per_week, process_CG, sum_hits_per_country, startTime_cast, yearweek, fullVisitorId, totals_transactions, sum_pageviews_per_fullVisitorId, date_cast, timeOnSite, avg_timeOnSite_per_fullVisitorId, process_OS, totalTR, visitNumber, sum_hits_per_fullVisitorId, revenue];;\n'Project [revenue#1218, totalTR#1219, process_CG#17019, process_browser#16314, process_country#3109, process_OS#15415, hits#1222, pageview#1223, visitNumber#1224, avg_timeOnSite_per_fullVisitorId#11808, sum_hits_per_day#3314L, sum_hits_per_week#3417L, sum_pageviews_per_country#3989L, sum_hits_per_country#4040L, sum_pageviews_per_fullVisitorId#5967L, sum_hits_per_fullVisitorId#8402L, 'features]\n+- Project [fullVisitorId#28, process_country#3109, revenue#1218, totalTR#1219, device_operatingSystem#45, device_browser#33, geoNetwork_country#52, channelGrouping#26, startTime_cast#1220, date_cast#2008, hits#1222, pageview#1223, visitNumber#1224, timeOnSite#1225, totals_transactions#1226, yearweek#2023, yearday#2039, sum_hits_per_day#3314L, sum_hits_per_week#3417L, sum_pageviews_per_country#3989L, sum_hits_per_country#4040L, sum_pageviews_per_fullVisitorId#5967L, sum_hits_per_fullVisitorId#8402L, avg_timeOnSite_per_fullVisitorId#11808, ... 3 more fields]\n +- Project [fullVisitorId#28, process_country#3109, revenue#1218, totalTR#1219, device_operatingSystem#45, device_browser#33, geoNetwork_country#52, channelGrouping#26, startTime_cast#1220, date_cast#2008, hits#1222, pageview#1223, visitNumber#1224, timeOnSite#1225, totals_transactions#1226, yearweek#2023, yearday#2039, sum_hits_per_day#3314L, sum_hits_per_week#3417L, sum_pageviews_per_country#3989L, sum_hits_per_country#4040L, sum_pageviews_per_fullVisitorId#5967L, sum_hits_per_fullVisitorId#8402L, avg_timeOnSite_per_fullVisitorId#11808, ... 2 more fields]\n +- Project [fullVisitorId#28, process_country#3109, revenue#1218, totalTR#1219, device_operatingSystem#45, device_browser#33, geoNetwork_country#52, channelGrouping#26, startTime_cast#1220, date_cast#2008, hits#1222, pageview#1223, visitNumber#1224, timeOnSite#1225, totals_transactions#1226, yearweek#2023, yearday#2039, 

In [29]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(DF_train)
model = pipelineModel.transform(DF_train)

In [30]:
from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x: (x["revenue"], DenseVector(x["features"])))

In [31]:
train = sqlContext.createDataFrame(input_data, ["revenue", "features"])	

In [32]:
display(train)

revenue,features
0.0,"List(1, 39, List(), List(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 41818.0, 219814.0, 44091.0, 52571.0, 1.0, 1.0))"
0.0,"List(1, 39, List(), List(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 1.0, 50.0, 25007.0, 175950.0, 91686.0, 106656.0, 3.0, 3.0))"
0.0,"List(1, 39, List(), List(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 8.0, 8.0, 1.0, 139.0, 11192.0, 70239.0, 3500525.0, 4267861.0, 8.0, 8.0))"
0.0,"List(1, 39, List(), List(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 1.0, 184.0, 28033.0, 150926.0, 3500525.0, 4267861.0, 4.0, 5.0))"
0.0,"List(1, 39, List(), List(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 25620.0, 150926.0, 3500525.0, 4267861.0, 2.0, 2.0))"
0.0,"List(1, 39, List(), List(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 0.0, 29005.0, 150926.0, 3500525.0, 4267861.0, 2.0, 2.0))"
0.0,"List(1, 39, List(), List(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 37.0, 27.0, 1.0, 613.0, 16201.0, 165057.0, 98659.0, 116957.0, 27.0, 37.0))"
0.0,"List(1, 39, List(), List(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 18310.0, 152034.0, 38101.0, 43647.0, 1.0, 1.0))"
0.0,"List(1, 39, List(), List(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 12.0, 22389.0, 144686.0, 175051.0, 198757.0, 2.0, 2.0))"
0.0,"List(1, 39, List(), List(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 24600.0, 186124.0, 3500525.0, 4267861.0, 1.0, 1.0))"


In [33]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
gbt = GBTRegressor(featuresCol="features", labelCol="revenue", maxIter = 15, maxDepth = 25)
train_data, test_data = train.randomSplit([.8,.2],seed=1234)
gbtModel = gbt.fit(train_data)

In [34]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
CrossValidator.setEstimator(pipeline)

# Let's tune over our dt.maxDepth parameter on the values 2 and 3, create a paramter grid using the ParamGridBuilder
maxDepth_grid = [20]
maxBins_grid = [30]
maxIter_grid = [15]
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, maxDepth_grid)
             .addGrid(gbt.maxBins, maxBins_grid)
             .addGrid(gbt.maxIter, maxIter_grid)
             .build())

# Add the grid to the CrossValidator
CrossValidator.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
gbtModel = crossval.fit(train_data).bestModel

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2487822865328498> in <module> () 
 1 from pyspark . ml . tuning import ParamGridBuilder , CrossValidator 
 ----> 2 CrossValidator . setEstimator ( pipeline ) 
 3 
 4 # Let's tune over our dt.maxDepth parameter on the values 2 and 3, create a paramter grid using the ParamGridBuilder 
 5 maxDepth_grid = [ 20 ] 

 TypeError : unbound method setEstimator() must be called with CrossValidator instance as first argument (got Pipeline instance instead)

In [35]:
# Make prediction
predictions = gbtModel.transform(test_data)
display(predictions.filter("revenue>0").select("revenue", "prediction"))

In [36]:
display(predictions.select("revenue", "prediction"))

In [37]:
# Now let's compute an evaluation metric for our test dataset
from pyspark.ml.evaluation import RegressionEvaluator

# Create an RMSE evaluator using the label and predicted columns
regEval = RegressionEvaluator(predictionCol="prediction", labelCol="revenue", metricName="rmse")

# Run the evaluator on the DataFrame
rmse = regEval.evaluate(predictions)
rmse